# Mediation analysis on CAD

In this script we will run a mediation analysis with coronary artery disease

## Preliminary steps

In [1]:
#### LIBRARIES
library(mediation)
library(stringr)

Loading required package: MASS

Loading required package: Matrix

Loading required package: mvtnorm

Loading required package: sandwich

mediation: Causal Mediation Analysis
Version: 4.5.0




In [2]:
#### PATHS
setwd('../')
mainpath <- getwd()
datapath <- paste0(mainpath, '/Data')

In [3]:
#### DATA
setwd(datapath)
snp_significant_met <- read.csv('MWAS_SNPs.txt', header=FALSE)
snp_gwas_cad        <- read.table('LURIC/results_Luric_cadyn_gwas.txt', header=TRUE) #GWAS on CAD
snp_met_pair        <- read.csv('results_genomewide_sig_LURIC_GenotypeMetabotype.csv') #mQTL
metabolites         <- read.table('LURIC/MWAS_metab_data.txt', header=TRUE, sep='\t') #Metabolite data

In [4]:
sprintf('There are %i SNPs significantly associated with metabolites', length(snp_significant_met[,1]))

[1] "There are 282 SNPs significantly associated with metabolites"

## Selecting SNPs

We will select SNPs based on the following criteria:
- From the SNPs that are significantly associated with metabolites, select the the top ones that are associated with CAD

In [5]:
#### SELECT SNPS FROM METABOLITES ASSOCIATION TO CAD ASSOCIATION
idx <- which(snp_gwas_cad$Var1_ID %in% snp_significant_met$V1)

#### SORT SNPs
sorted      <- sort.int(snp_gwas_cad[idx, 'Var1_Pval'], index.return = TRUE)
temp_snps   <- snp_gwas_cad[idx,]
sorted_snps <- temp_snps[sorted$ix, ]

In [6]:
sprintf('The highest GWAS pvalue on CAD is %f, and the lowest one is %f, so we will select SNPs with p < 0.05', max(sorted_snps$Var1_Pval), min(sorted_snps$Var1_Pval))
b           <- sorted_snps$Var1_Pval < 0.05
sorted_snps <- sorted_snps[b,]
sorted_snps

[1] "The highest GWAS pvalue on CAD is 0.993537, and the lowest one is 0.011338, so we will select SNPs with p < 0.05"

,Var1_ID,Var1_Pos,Var1_Pval
,<chr>,<chr>,<dbl>
6231,rs11240099,1:144973250,0.0113379
9507,rs2353029,17:37280697,0.0169575
10944,rs11965495,6:90719014,0.0194511
13448,rs992375,11:33898323,0.0237699
16456,rs12170368,22:31441247,0.0291719
17579,rs17830422,15:86510061,0.0311222
20367,rs5749498,22:31436947,0.0359842
22720,rs1419373,9:21047062,0.0400954


In [7]:
#### GET SNP METABOLITE PAIR
b            <- which(snp_met_pair$Var1_ID %in% sorted_snps$Var1_ID)
snp_met_pair <- snp_met_pair[b,]
snp_met_pair

,Outcome,Var1_ID
,<chr>,<chr>
72,pc_42_4,rs12170368
79,hydro_7,rs11965495
104,pc_42_4,rs5749498
131,stigmast,rs2353029
162,pc_o_21,rs992375
173,spm_di5,rs1419373
253,Kynurenic_acid,rs17830422
297,spm_28_3,rs1419373
328,pc_o_21,rs5749498


## Export selected SNP only

We will extract and read the selection of SNPs

In [8]:
setwd(datapath)
write.table(sorted_snps$Var1_ID, file = 'LURIC/top_snps_cad_met.txt', quote = FALSE, row.names = FALSE, col.names = FALSE)

In [9]:
plink_command <- 'plink --bfile LURIC/LURIC_idpat_final_relRem_geno99_mind99_maf05 --extract LURIC/top_snps_cad_met.txt --recodeA --out LURIC/LURIC_idpat_final_relRem_geno99_mind99_maf05_topSNPsCAD_MET'
system(plink_command)

In [10]:
#### READ SNP FILES
setwd(datapath)
snps <- read.table('LURIC/LURIC_idpat_final_relRem_geno99_mind99_maf05_topSNPsCAD_MET.raw', header=TRUE) #genotypes

## Mediation Analysis

We will do pairwise SNP-metabolite mediation analysis

In [11]:
#### INTERSECT DATA SETS
dat <- merge(metabolites, snps, by.x='idpat', by.y='IID')

for(i in 1:length(snp_met_pair[,1])){
    metname <- snp_met_pair[i,1]
    snpname <- snp_met_pair[i,2]
    idx     <- which(str_detect(colnames(dat), snpname))
    dat_for_fit <- cbind(dat$cadyn, dat[,idx], scale(dat[,metname]), scale(dat$age), scale(dat$bmi), scale(dat$waihip), dat$sex )
    dat_for_fit <- na.omit(dat_for_fit)
    
    #### EFFECT ON MEDIATOR
    fit.mediator <- lm(dat_for_fit[,3] ~ dat_for_fit[,2] + dat_for_fit[,4] + dat_for_fit[,5] + dat_for_fit[,6] + dat_for_fit[,7])
    #summary(fit.mediator)
    
    #### EFFECT ON OUTCOME
    fit.outcome <- lm(dat_for_fit[,1] ~ dat_for_fit[,2] + dat_for_fit[,3] + dat_for_fit[,4] + dat_for_fit[,5] + dat_for_fit[,6] + dat_for_fit[,7])
    #summary(fit.outcome)
    
    #### MEDIATION
    fit.mediation <- mediate(fit.mediator, fit.outcome, treat = 'dat_for_fit[, 2]', mediator = 'dat_for_fit[, 3]')
    print(snp_met_pair[i,])
    print(summary(fit.mediation))
    cat('\n')
}


   Outcome    Var1_ID
72 pc_42_4 rs12170368

Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME            -0.0006      -0.0032         0.00    0.58
ADE             -0.0151      -0.0608         0.03    0.51
Total Effect    -0.0157      -0.0618         0.03    0.49
Prop. Mediated   0.0122      -0.4169         0.53    0.73

Sample Size Used: 2812 


Simulations: 1000 


   Outcome    Var1_ID
79 hydro_7 rs11965495

Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME            0.00348     -0.00304         0.01    0.33
ADE            -0.02640     -0.07830         0.03    0.35
Total Effect   -0.02292     -0.07547         0.03    0.41
Prop. Mediated -0.07298     -1.31890         1.88    0.61

Sample Size Used: 1979 


Simulations: 1000 


    Outcome   Var1_ID
104 pc_42_4 rs5749498

Causal Mediation Analysis 

Quasi-Bayesian Confidence Inter